In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import pathlib
from util import runGridSearchClassifiers

## Loading data

In [3]:
directory_path = '../../04_-_Dev/videos'
features = 'emobase'#''

In [ ]:
currentDirectory = pathlib.Path(directory_path)
currentPattern = "*." + features + ".annotated.csv"
file_list = [str(currentFile) for currentFile in currentDirectory.glob(currentPattern)]

df_total = pd.DataFrame()
for filename in file_list:
    df = pd.read_csv(filename, delimiter=';')
    df_total = pd.concat([df_total, df], axis=0)

print('Number of videos', df_total['video_name'].nunique())
print('Number of annotations', df_total[['video_name','diapo']].drop_duplicates().shape[0])


In [2]:
df_total = df_total.dropna(axis=0)

NameError: name 'df_total' is not defined

In [9]:
df_total.to_pickle(directory_path + '/audio_' + features + '_data.p')

In [4]:
df_total = pd.read_pickle(directory_path + '/audio_' + features + '_data.p')

## Data pre-processing 1 - Sans utilisation de la temporalité

In [11]:
df_total

,frameIndex,frameTime,pcm_intensity_sma,pcm_loudness_sma,mfcc_sma[1],mfcc_sma[2],mfcc_sma[3],mfcc_sma[4],mfcc_sma[5],mfcc_sma[6],...,pcm_zcr_sma_de,voiceProb_sma_de,F0_sma_de,F0env_sma_de,video_name,stress_global,type_candidat,sexe,stress,diapo
0,0,0.00,0.000000e+00,0.000000,-1.839101,2.378769,-2.217382,1.524796,-0.692341,0.101261,...,-0.000242,0.016226,0.0,0.0,WIN_20210409_10_26_11_Pro,0.0,Stagiaire,F,1.0,1
1,1,0.01,0.000000e+00,0.000000,-1.839101,2.378769,-2.217382,1.524796,-0.692341,0.101261,...,-0.000393,0.007565,0.0,0.0,WIN_20210409_10_26_11_Pro,0.0,Stagiaire,F,1.0,1
2,2,0.02,0.000000e+00,0.000000,-5.952965,6.019684,-3.599674,1.358609,-0.693720,0.796165,...,-0.000272,0.011002,0.0,0.0,WIN_20210409_10_26_11_Pro,0.0,Stagiaire,F,1.0,1
3,3,0.03,0.000000e+00,0.000000,-4.113864,3.640915,-1.382292,-0.166187,-0.001379,0.694905,...,0.000000,0.009354,0.0,0.0,WIN_20210409_10_26_11_Pro,0.0,Stagiaire,F,1.0,1
4,4,0.04,0.000000e+00,0.000000,-5.911817,5.975944,-3.576790,1.367325,-0.725028,0.827764,...,0.000272,0.007488,0.0,0.0,WIN_20210409_10_26_11_Pro,0.0,Stagiaire,F,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78139,78139,781.39,8.054489e-12,0.029190,-19.760860,6.082466,-2.668031,5.229023,3.715201,-1.398331,...,-0.002025,0.012437,0.0,0.0,WIN_20210416_08_06_54_Pro,1.0,Stagiaire,F,1.0,18
78140,78140,781.40,4.319410e-12,0.023441,-21.733470,5.059051,-2.391827,4.356356,5.255848,-0.879569,...,-0.003778,0.021489,0.0,0.0,WIN_20210416_08_06_54_Pro,1.0,Stagiaire,F,1.0,18
78141,78141,781.41,3.975198e-12,0.022989,-21.913380,3.731894,-4.589119,0.219700,1.376662,-5.050643,...,-0.005409,0.040231,0.0,0.0,WIN_20210416_08_06_54_Pro,1.0,Stagiaire,F,1.0,18
78142,78142,781.42,3.662190e-12,0.022472,-22.256340,2.845562,-4.491580,2.111024,3.834707,-3.251046,...,-0.007011,0.052342,0.0,0.0,WIN_20210416_08_06_54_Pro,1.0,Stagiaire,F,1.0,18


In [12]:
from scipy.stats import kurtosis, skew

def percentil25(x): 
    return np.percentile(x, q=25)

def percentil75(x): 
    return np.percentile(x, q=75)

In [13]:
# Sequence as a whole with 9 agregators
X = df_total.iloc[:,2:].groupby(['video_name','diapo']).agg({'mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew}).iloc[:,:-18]

In [14]:
# Sequence as a whole with 4 agregators
#X = df_total.iloc[:,2:].groupby(['video_name','diapo']).agg({'mean','std', kurtosis, skew}).iloc[:,:-8]

In [15]:
X.shape

(240, 468)

In [16]:
X

pcm_intensity_sma                             \
                                           median   percentil25     kurtosis   
video_name                diapo                                                
Test_pour_AFPA            1          6.973596e-11  3.821638e-11  3147.223384   
                          8          1.126761e-08  7.489974e-11   123.403605   
                          9          2.440317e-09  6.744114e-11   806.978682   
                          10         7.152661e-10  5.894842e-11   796.376223   
                          11         5.700818e-09  7.155228e-11   827.561300   
...                                           ...           ...          ...   
WIN_20210417_14_53_12_Pro 10         2.086603e-13  0.000000e+00    57.749166   
                          11         6.047051e-11  0.000000e+00    38.423855   
                          12         2.128377e-12  1.669329e-13   151.489483   
                          17         2.809062e-08  4.173518e-14    28.893323   
                          18         3.088259e-12  0.000000e+00  1139.092180   

                                                                           \
                                          max           std           min   
video_name                diapo                                             
Test_pour_AFPA            1      1.795643e-05  2.748470e-07  3.338736e-13   
                          8      2.552147e-05  1.159030e-06  2.629277e-12   
                          9      4.923681e-05  1.142409e-06  1.293720e-12   
                          10     5.260212e-05  1.367874e-06  1.544100e-12   
                          11     6.543399e-05  1.446588e-06  8.346372e-13   
...                                       ...           ...           ...   
WIN_20210417_14_53_12_Pro 10     6.300619e-06  4.197754e-07  0.000000e+00   
                          11     5.826240e-06  3.674674e-07  0.000000e+00   
                          12     9.125553e-09  3.760241e-10  0.000000e+00   
                          17     8.491340e-06  7.997970e-07  0.000000e+00   
                          18     1.093041e-08  2.102153e-10  0.000000e+00   

                                                                        \
                                         mean   percentil75       skew   
video_name                diapo                                          
Test_pour_AFPA            1      1.137963e-08  1.225284e-10  52.105435   
                          8      3.445894e-07  1.725128e-07   8.773747   
                          9      2.626492e-07  1.095324e-07  21.745491   
                          10     2.443886e-07  9.802079e-08  23.775476   
                          11     2.895271e-07  1.422769e-07  23.492521   
...                                       ...           ...        ...   
WIN_20210417_14_53_12_Pro 10     1.405645e-07  6.072907e-08   6.260282   
                          11     1.452637e-07  1.018110e-07   4.954644   
                          12     9.356331e-11  2.517556e-11  10.185306   
                          17     3.458074e-07  3.321533e-07   4.658821   
                          18     4.188290e-11  3.496205e-11  28.084717   

                                pcm_loudness_sma  ... F0_sma_de F0env_sma_de  \
                                          median  ...      skew       median   
video_name                diapo                   ...                          
Test_pour_AFPA            1             0.050654  ...  0.299397          0.0   
                          8             0.224705  ...  0.226955          0.0   
                          9             0.141097  ...  0.105952          0.0   
                          10            0.097588  ...  0.263135          0.0   
                          11            0.182440  ...  0.101022          0.0   
...                                          ...  ...       ...          ...   
WIN_20210417_14_53_12_Pro 10            0.006974  ... -0.001834          0.

In [17]:
X_temp = df_total[['video_name','diapo','type_candidat','sexe']].groupby(['video_name','diapo']).agg({'type_candidat':'first','sexe':'first'})
X_temp.loc[X_temp['type_candidat'] == 'Stagiaire','Stagiaire'] = 1
X_temp.loc[X_temp['type_candidat'] != 'Stagiaire','Stagiaire'] = 0
X_temp.loc[X_temp['sexe'] == 'F','Femme'] = 1
X_temp.loc[X_temp['sexe'] != 'F','Femme'] = 0
X_temp = X_temp.drop(['type_candidat','sexe'], axis=1)

In [19]:
X = pd.concat([X,X_temp],axis=1)

In [20]:
y = df_total.iloc[:,2:].groupby(['video_name','diapo']).agg({'stress':'mean'}).iloc[:,-1]

In [21]:
X.shape, y.shape

((240, 470), (240,))

In [22]:
X.to_pickle(directory_path + '/audio_' + features + '_data_X.p')
y.to_pickle(directory_path + '/audio_' + features + '_data_y.p')

In [23]:
from scipy.stats import kurtosis, skew
X_audio = df_total.loc[df_total['diapo'].isin([8, 9, 10, 11, 17]),:].iloc[:,2:].groupby(['video_name','diapo']).agg({'mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew}).iloc[:,:-18]
y_audio = df_total.loc[df_total['diapo'].isin([8, 9, 10, 11, 17]),:].iloc[:,2:].groupby(['video_name','diapo']).agg({'stress':'mean'}).iloc[:,-1]

X_audio.to_pickle(directory_path + '/audio_' + features + '_data_X_audio.p')
y_audio.to_pickle(directory_path + '/audio_' + features + '_data_y_audio.p')

In [24]:
X_audio.shape

(150, 468)

## Modèles

In [3]:
X = pd.read_pickle(directory_path + '/audio_' + features + '_data_X.p')
y = pd.read_pickle(directory_path + '/audio_' + features + '_data_y.p')

X_audio = pd.read_pickle(directory_path + '/audio_' + features + '_data_X_audio.p')
y_audio = pd.read_pickle(directory_path + '/audio_' + features + '_data_y_audio.p')

In [4]:
X.shape, X_audio.shape

((240, 470), (150, 468))

In [27]:
X

(pcm_intensity_sma, median)  \
video_name                diapo                                
Test_pour_AFPA            1                     6.973596e-11   
                          8                     1.126761e-08   
                          9                     2.440317e-09   
                          10                    7.152661e-10   
                          11                    5.700818e-09   
...                                                      ...   
WIN_20210417_14_53_12_Pro 10                    2.086603e-13   
                          11                    6.047051e-11   
                          12                    2.128377e-12   
                          17                    2.809062e-08   
                          18                    3.088259e-12   

                                 (pcm_intensity_sma, percentil25)  \
video_name                diapo                                     
Test_pour_AFPA            1                          3.821638e-11   
                          8                          7.489974e-11   
                          9                          6.744114e-11   
                          10                         5.894842e-11   
                          11                         7.155228e-11   
...                                                           ...   
WIN_20210417_14_53_12_Pro 10                         0.000000e+00   
                          11                         0.000000e+00   
                          12                         1.669329e-13   
                          17                         4.173518e-14   
                          18                         0.000000e+00   

                                 (pcm_intensity_sma, kurtosis)  \
video_name                diapo                                  
Test_pour_AFPA            1                        3147.223384   
                          8                         123.403605   
                          9                         806.978682   
                          10                        796.376223   
                          11                        827.561300   
...                                                        ...   
WIN_20210417_14_53_12_Pro 10                         57.749166   
                          11                         38.423855   
                          12                        151.489483   
                          17                         28.893323   
                          18                       1139.092180   

                                 (pcm_intensity_sma, max)  \
video_name                diapo                             
Test_pour_AFPA            1                  1.795643e-05   
                          8                  2.552147e-05   
                          9                  4.923681e-05   
                          10                 5.260212e-05   
                          11                 6.543399e-05   
...                                                   ...   
WIN_20210417_14_53_12_Pro 10                 6.300619e-06   
                          11                 5.826240e-06   
                          12                 9.125553e-09   
                          17                 8.491340e-06   
                          18                 1.093041e-08   

                                 (pcm_intensity_sma, std)  \
video_name                diapo                             
Test_pour_AFPA            1                  2.748470e-07   
                          8                  1.159030e-06   
                          9                  1.142409e-06   
                          10                 1.367874e-06   
                          11                 1.446588e-06   
...                                                   ...   
WIN_20210417_14_53_12_Pro 10                 4.197754e-07   
                          11                 3.674674e-07   
                          12           

In [28]:
X = X.fillna(0)

### Leave one interview out
#### All diapos

In [7]:
diapo_selection = '_all'

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut

models_list = [RandomForestClassifier(random_state = 42, n_jobs=-1)]
parameters_list = [
                {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20, 25], 'class_weight':[None,'balanced']}
                ]
groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [6]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 64 candidates, totalling 1920 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   41.8s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1920 out of 1920 | elapsed:  3.5min finished
Best estimator RandomForestClassifier(max_depth=20, n_jobs=-1, random_state=42)
Best results 0.48991839179339175
Best params {'class_weight': None, 'max_depth': 20, 'n_estimators': 100}
accuracy (mean, std) 0.49583333333333335 0.21525985588482482
f1 (mean, std) 0.48991839179339175 0.2336572988754913
balanced accuracy (mean, std) 0.4811375661375662 0.233584357587154
precision (mean, std) 0.6014980158730159 0.28

In [8]:
# Saving predictions
df_ypredict = pd.concat([X.reset_index()[['video_name','diapo']],pd.DataFrame(y_predict, columns=['ypredict'])],axis=1)
df_ypredict.to_csv('ypredict_' + features + '_diapo' + diapo_selection + '.csv')

### Audio diapos only

In [9]:
diapo_selection = '_audio_only'

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut

models_list = [RandomForestClassifier(random_state = 42, n_jobs=-1)]
parameters_list = [
                {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6,7 ,8,9, 10, 15, 20, 25], 'class_weight':[None,'balanced']}
                ]
groups = X_audio.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X_audio, y_audio, groups)

In [12]:
best_result, y_predict, result_list = runGridSearchClassifiers(X_audio, y_audio, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 88 candidates, totalling 2640 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   39.9s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 2640 out of 2640 | elapsed:  4.3min finished
Best estimator RandomForestClassifier(class_weight='balanced', max_depth=6, n_jobs=-1,
                       random_state=42)
Best results 0.5235608465608466
Best params {'class_weight': 'balanced', 'max_depth': 6, 'n_estimators': 100}
accuracy (mean, std) 0.5400000000000001 0.2973213749463702
f1 (mean, std) 0.5235608465608466 0.3135840001372412
b

In [13]:
df_ypredict = pd.concat([X_audio.reset_index()[['video_name','diapo']],pd.DataFrame(y_predict, columns=['ypredict'])],axis=1)
df_ypredict.to_csv('ypredict_' + features + '_diapo' + diapo_selection + '.csv')

## Stress global
### All diapos

In [60]:
diapo_selection = '_all'

In [61]:
df_ypredict = pd.read_csv('ypredict_' + features + '_diapo' + diapo_selection + '.csv')
df_ypredict = df_ypredict.drop(df_ypredict.columns[0],axis=1)
df_ypredict.columns = ['video_name','diapo','ypredict']
ypredict_stress_diapo = df_ypredict.pivot_table(values='ypredict', columns='diapo',index='video_name',aggfunc='mean')

In [62]:
#df_annotations_stress_diapo = df_total.pivot_table(values='stress', index='video_name',columns='diapo', aggfunc='mean')
#df_annotations_stress_global = df_total.pivot_table(values='stress_global', index='video_name', aggfunc='mean')
#df_annotations_stress = df_annotations_stress_diapo.merge(df_annotations_stress_global, on='video_name')
#df_annotations_stress

In [63]:
df_annotations_stress = pd.read_csv('annotations.csv')

#### En utilisant les annotations comme X

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                KNeighborsClassifier(),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2], 'class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]
X = df_annotations_stress.iloc[:,:-1].set_index('video_name')
y = df_annotations_stress.iloc[:,-1]
cv = 5

In [65]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.1, multi_class='multinomial', random_state=42)
Best results 0.7651587301587301
Best params {'C': 0.1, 'class_weight': None}
accuracy (mean, std) 0.8 0.0666666666666667
f1 (mean, std) 0.7651587301587301 0.11176124986563704
balanced accuracy (mean, std) 0.8 0.0666666666666667
precision (mean, std) 0.8016666666666665 0.16758635142870432
recall (mean, std) 0.8 0.0666666666666667

Fitting 5 folds for each of 44 candidates, totalling 220 fits
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 220 out of 220 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator KNeighborsCla

In [66]:
ypredict_stress_global = best_result['best_estimator'].predict(ypredict_stress_diapo)
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score, precision_score, recall_score
print('Accuracy',accuracy_score(y.values,ypredict_stress_global))
print('F1',f1_score(y.values,ypredict_stress_global, average='weighted'))
print('Balanced accuracy',balanced_accuracy_score(y.values,ypredict_stress_global))
print('Precision',precision_score(y.values,ypredict_stress_global, average='weighted'))
print('Recall',recall_score(y.values,ypredict_stress_global, average='weighted'))

Accuracy 0.36666666666666664
F1 0.3148148148148148
Balanced accuracy 0.33888888888888885
Precision 0.28431372549019607
Recall 0.36666666666666664


In [ ]:
pd.concat([X.reset_index().iloc[:,0],y, pd.DataFrame(ypredict_stress_global,columns=['predicted_stress_global'])],axis=1)

#### En utilisant les prédictions comme annotations

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                KNeighborsClassifier(),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]
X = ypredict_stress_diapo
y = df_annotations_stress.iloc[:,-1]
groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [71]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=2, multi_class='multinomial', random_state=42)
Best results 0.3683333333333333
Best params {'C': 2, 'class_weight': None}
accuracy (mean, std) 0.39999999999999997 0.1699673171197595
f1 (mean, std) 0.3683333333333333 0.1377912179824477
balanced accuracy (mean, std) 0.38888888888888884 0.18592445034090566
precision (mean, std) 0.37111111111111106 0.14476033833738974
recall (mean, std) 0.39999999999999997 0.1699673171197595

Fitting 5 folds for each of 44 candidates, totalling 220 fits
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 220 out of 220 | elapsed:    0.6s finis

In [72]:
best_result

{'best_estimator': LogisticRegression(C=2, multi_class='multinomial', random_state=42),
 'best_score': 0.3683333333333333,
 'best_params': {'C': 2, 'class_weight': None},
 'mean_test_f1_score': 0.3683333333333333,
 'std_test_f1_score': 0.1377912179824477,
 'mean_test_accuracy_score': 0.39999999999999997,
 'std_test_accuracy_score': 0.1699673171197595,
 'mean_test_balanced_accuracy_score': 0.38888888888888884,
 'std_test_balanced_accuracy_score': 0.18592445034090566,
 'mean_test_precision': 0.37111111111111106,
 'std_test_precision': 0.14476033833738974,
 'mean_test_recall': 0.39999999999999997,
 'std_test_recall': 0.1699673171197595}

### Audio diapos only

In [73]:
diapo_selection = '_audio_only'

In [74]:
df_ypredict = pd.read_csv('ypredict_' + features + '_diapo' + diapo_selection + '.csv')
df_ypredict = df_ypredict.drop(df_ypredict.columns[0],axis=1)
df_ypredict.columns = ['video_name','diapo','ypredict']
ypredict_stress_diapo = df_ypredict.pivot_table(values='ypredict', columns='diapo',index='video_name',aggfunc='mean')

#### En utilisant les annotations comme X

In [82]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                KNeighborsClassifier(),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2], 'class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]
X = df_annotations_stress[['video_name','8','9','10','11','17','stress_global']].iloc[:,:-1].set_index('video_name')
y = df_annotations_stress.iloc[:,-1]
cv = 5

In [83]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  60 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.1, class_weight='balanced', multi_class='multinomial',
                   random_state=42)
Best results 0.7092063492063492
Best params {'C': 0.1, 'class_weight': 'balanced'}
accuracy (mean, std) 0.7666666666666667 0.08164965809277264
f1 (mean, std) 0.7092063492063492 0.134722169623853
balanced accuracy (mean, std) 0.7666666666666667 0.08164965809277264
precision (mean, std) 0.7166666666666666 0.20042393341719386
recall (mean, std) 0.7666666666666667 0.08164965809277264

Fitting 5 folds for each of 44 candidates, totalling 220 fits
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.2

In [85]:
ypredict_stress_global = best_result['best_estimator'].predict(ypredict_stress_diapo)
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score, precision_score, recall_score
print('Accuracy',accuracy_score(y.values,ypredict_stress_global))
print('F1',f1_score(y.values,ypredict_stress_global, average='weighted'))
print('Balanced accuracy',balanced_accuracy_score(y.values,ypredict_stress_global))
print('Precision',precision_score(y.values,ypredict_stress_global, average='weighted'))
print('Recall',recall_score(y.values,ypredict_stress_global, average='weighted'))

Accuracy 0.5666666666666667
F1 0.5666753812636165
Balanced accuracy 0.5555555555555555
Precision 0.573005698005698
Recall 0.5666666666666667


#### En utilisant les prédictions des diapos

In [86]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                KNeighborsClassifier(),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2], 'class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]
X = ypredict_stress_diapo
y = df_annotations_stress.iloc[:,-1]
groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [87]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 12 candidates, totalling 360 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:    1.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.01, class_weight='balanced', multi_class='multinomial',
                   random_state=42)
Best results 0.4666666666666667
Best params {'C': 0.01, 'class_weight': 'balanced'}
accuracy (mean, std) 0.4666666666666667 0.49888765156985887
f1 (mean, std) 0.4666666666666667 0.49888765156985887
balanced accuracy (mean, std) 0.4666666666666667 0.49888765156985887
precision (mean, std) 0.4666666666666667 0.49888765156985887
recall (mean, std) 0.4666666666666667 0.49888765156985887

Fitting 30 folds for each of 44 candidates, totalling 1320 fits
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.2s
[Parallel

In [88]:
best_result

{'best_estimator': LogisticRegression(C=0.01, class_weight='balanced', multi_class='multinomial',
                    random_state=42),
 'best_score': 0.4666666666666667,
 'best_params': {'C': 0.01, 'class_weight': 'balanced'},
 'mean_test_f1_score': 0.4666666666666667,
 'std_test_f1_score': 0.49888765156985887,
 'mean_test_accuracy_score': 0.4666666666666667,
 'std_test_accuracy_score': 0.49888765156985887,
 'mean_test_balanced_accuracy_score': 0.4666666666666667,
 'std_test_balanced_accuracy_score': 0.49888765156985887,
 'mean_test_precision': 0.4666666666666667,
 'std_test_precision': 0.49888765156985887,
 'mean_test_recall': 0.4666666666666667,
 'std_test_recall': 0.49888765156985887}

## Aggregation all frames within the video to predict the global stress

#### All diapo

In [26]:
from scipy.stats import kurtosis, skew

def percentil25(x): 
    return np.percentile(x, q=25)

def percentil75(x): 
    return np.percentile(x, q=75)

X = df_total.iloc[:,2:].groupby(['video_name']).agg({'mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew}).iloc[:,:-27]

In [27]:
y = df_total.iloc[:,2:].groupby(['video_name']).agg({'stress_global':'mean'}).iloc[:,-1]

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42))])

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                pipe,
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [34]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 20 candidates, totalling 600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:   12.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.5, class_weight='balanced', multi_class='multinomial',
                   random_state=42)
Best results 0.23333333333333334
Best params {'C': 0.5, 'class_weight': 'balanced'}
accuracy (mean, std) 0.23333333333333334 0.42295258468165065
f1 (mean, std) 0.23333333333333334 0.42295258468165065
balanced accuracy (mean, std) 0.23333333333333334 0.42295258468165065
precision (mean, std) 0.23333333333333334 0.42295258468165065
recall (mean, std) 0.23333333333333334 0.42295258468165065

Fitting

In [35]:
best_result

{'best_estimator': KNeighborsClassifier(n_neighbors=12, weights='distance'),
 'best_score': 0.5,
 'best_params': {'n_neighbors': 12, 'p': 2, 'weights': 'distance'},
 'mean_test_f1_score': 0.5,
 'std_test_f1_score': 0.5,
 'mean_test_accuracy_score': 0.5,
 'std_test_accuracy_score': 0.5,
 'mean_test_balanced_accuracy_score': 0.5,
 'std_test_balanced_accuracy_score': 0.5,
 'mean_test_precision': 0.5,
 'std_test_precision': 0.5,
 'mean_test_recall': 0.5,
 'std_test_recall': 0.5}

#### Audio diapo only

In [36]:
from scipy.stats import kurtosis, skew

def percentil25(x): 
    return np.percentile(x, q=25)

def percentil75(x): 
    return np.percentile(x, q=75)

X_audio = df_total[df_total.diapo.isin([8,9,10,11,17])].iloc[:,2:].groupby(['video_name']).agg({'mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew}).iloc[:,:-27]

In [37]:
y = df_total.iloc[:,2:].groupby(['video_name']).agg({'stress_global':'mean'}).iloc[:,-1]

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42))])

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                pipe,
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]
X = X_audio
groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [42]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 20 candidates, totalling 600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:   11.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.5, multi_class='multinomial', random_state=42)
Best results 0.4666666666666667
Best params {'C': 0.5, 'class_weight': None}
accuracy (mean, std) 0.4666666666666667 0.49888765156985887
f1 (mean, std) 0.4666666666666667 0.49888765156985887
balanced accuracy (mean, std) 0.4666666666666667 0.49888765156985887
precision (mean, std) 0.4666666666666667 0.49888765156985887
recall (mean, std) 0.4666666666666667 0.49888765156985887

Fitting 30 folds for each of 80 candidates, totalling 2400 fits

In [43]:
best_result

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=1)),
                 ('logistic',
                  LogisticRegression(C=0.01, class_weight='balanced',
                                     multi_class='multinomial',
                                     random_state=42))]),
 'best_score': 0.6,
 'best_params': {'logistic__C': 0.01,
  'logistic__class_weight': 'balanced',
  'pca__n_components': 1},
 'mean_test_f1_score': 0.6,
 'std_test_f1_score': 0.4898979485566357,
 'mean_test_accuracy_score': 0.6,
 'std_test_accuracy_score': 0.4898979485566357,
 'mean_test_balanced_accuracy_score': 0.6,
 'std_test_balanced_accuracy_score': 0.4898979485566357,
 'mean_test_precision': 0.6,
 'std_test_precision': 0.4898979485566357,
 'mean_test_recall': 0.6,
 'std_test_recall': 0.4898979485566357}